<a href="https://colab.research.google.com/github/Sudeeppp-Mishra/LeafLens/blob/main/LeafLens_Train_on_DataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeafLens: AI-Based Plant Disease Detection

LeafLens classifies plant leaf images into healthy or diseased categories.
We use a pretrained ResNet18 model (transfer learning) for accuracy and faster training.

**Libraries used:**
- PIL / OpenCV / NumPy: Image loading and preprocessing
- PyTorch / Torchvision: Model training and evaluation
- Matplotlib: Visualization of metrics
- PySide6 / pyttsx3 / gTTS: GUI + audio output (later integration)

**Dataset:** PlantVillage (Image Classification)  
**Platform:** Google Colab (GPU)